# December 12, 2023

https://adventofcode.com/2023/day/12

In [1]:
import re

In [2]:
def parse_input( lines ):
    data = []
    groups = []
    for line in lines:
        d, g = line.split()
        g = [int(x) for x in g.split(",")]
        data.append(d)
        groups.append(g)

    return {'data': data, 'groups':groups}


In [3]:
test_str = f'''???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1'''
test = parse_input( test_str.split("\n") )

In [119]:
[x for x in "...??....???.????.?????".split( "." ) if len(x)>0]

['??', '???', '????', '?????']

In [4]:
fn = "data/12.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz = parse_input( [x.strip() for x in text] )

In [124]:
for i in range(10,4):
    print(i)

### Part 1

In [151]:
def divvy_groups( text, groups, min_pos = 0, verbose=False ):
    '''
    Determine how many ways to put the groups into the text (string vec)
    '''

    print("new call", text, groups, min_pos )
    # base case: no more groups, so make sure there are no unaccounted for #s
    if len(groups) == 0:
        if any( ["#" in t for t in text[min_pos:]] ):
            if verbose:
                print("<no soln>")
            return 0
        else:
            if verbose:
                print("success!")
            return 1
        
    if len(text) == 0:
        if verbose:
            print("out of room")
        return 0


    # determine last possible position to start the first group
    # needs to have enough room to fit the group
    # and can't skip over any #
    max_pos = len(text[0]) - groups[0]
    m = re.search("#", text[0][min_pos:])
    if m is not None:
        max_pos = min(max_pos, m.span()[0])


    tot_solutions = 0
    print("min", min_pos, "max", max_pos)
    for pos in range(min_pos, max_pos+1):
        print(text, groups, pos)

        next_pos = pos + groups[0] + 1
        # If the next char is #, then the group doesn't fit here
        if next_pos < len(text[0]) and text[0][next_pos] == "#":
            continue

        # given this position, find the number of solutions for remaining groups
        tot_solutions += divvy_groups( text, groups[1:], min_pos = pos + groups[0] + 1, verbose=verbose )

    tot_solutions += divvy_groups( text[1:], groups, min_pos=0, verbose=verbose)

    return tot_solutions

def count_solutions( text, groups, verbose=False ):
    text_vec = [x for x in text.split(".") if len(x) > 0]
    return divvy_groups( text_vec, groups, verbose=verbose)

In [216]:
m = re.search( "[^#]*?(\?*)(#.*?)(\?*)(?!.*#.*)", "?.?.#??.??.?#.?")#?#?????#??????????????????" )
m

<re.Match object; span=(0, 13), match='?.?.#??.??.?#'>

In [218]:
m[1], m[2], m[3],

('', '#??.??.?#', '')

In [ ]:
if 

In [168]:
count_solutions( "?.??.???.????.?????.???", [1] )

18

In [249]:
def encode_state( text, groups ):
    # assume text and groups are both non-empty

    gt = ":".join( [str(x) for x in groups] )
    return text + ":" + gt

def count_solutions( text, groups, memo=None, verbose=False ):
    '''
    text: string for this row of the map
    groups: sizes of broken strings to search for
    '''
    if verbose:
        print("Remaining string:", text)
        print(groups)    
   
    # BASE CASE:    
    # No groups: then if there are no #, there's a unique solution to make all ? a .
    if len(groups) == 0:
        if "#" in text:
            return 0
        else:
            return 1
    
    # We have at least 1 group, but no text to match it to!
    if len(text) <= 0:
        return 0
    
    # One group: if there are no #, then count the number of large enough ??? groups
    # otherwise, do a trickier solve
    if len(groups) == 1:
        m = re.search( "(\?*)(#+)(\?*)", text )
        if m is None:
            qmarks = re.findall( f'''\?{{{groups[0]},}}''', text )
            return sum( [len(x) - groups[0] + 1 for x in qmarks] )
        else:
            # This pattern contains 3 captures
            # 2nd is the shortest substring possible that contains all the #s
            # 1st is the possibly empty string of ?s immediately before the 2nd
            # 3rd is the possibly empty string of ?s immediately after the 2nd

            magic = re.search( "[^#]*?(\?*)(#.*?)(\?*)(?!.*#.*)", text )
            # If there is a working spring between the first and last #, then there is no solution
            if "." in magic[2]:
                return 0
            
            # how many ?s we can and must use in addition to the "core" #????#
            flex = groups[0] - len(magic[2])
            if flex < 0:
                # There are too many spaces between the first and last # and this group isn't big enough
                return 0

            n_solutions = 0
            for i in range( min(flex, len(magic[1])) + 1):
                if len(magic[3]) >= flex - i:
                    n_solutions += 1
            return n_solutions

    # First trim out the initial .s
    m = re.search("^\.*", text)
    if m is not None:
        text = text[ m.span()[1]: ]      

    if len(text) == 0:
        # no solutions because we're out of text and have unmatched groups
        return 0
    
    # CHECK MEMO to see if we've finished this problem before
    state = encode_state( text, groups )
    if memo is None:
        memo = dict()
    else:
        if state in memo.keys():
            return memo[state]
        
    
    # Now we can assume that the first char is ? or #
    # check for initial string of G ?s or #s
    pattern = f'''^[\?#]{{{groups[0]}}}(?!#)'''

    tot_solutions = 0
    while len(text) >= sum(groups) + len(groups) - 1:
        m = re.search( pattern, text )
        if m is not None:
            # found a match!
            # now solve the smaller problem with one less group
            tot_solutions += count_solutions( text[ groups[0]+1 :], groups[1:], memo, verbose=verbose )
        
        # if first char is #, then we can't advance cursor and try again
        if text[0] == "#":
            break

        # otherwise, trim leading ? and try again
        text = text[1:]         

    memo[state] = tot_solutions
    return tot_solutions




In [250]:
def part1( puzz ):
    tot = 0
    i = 0
    for d,g in zip(puzz['data'], puzz['groups']):
        tot += count_solutions( d, g )
        print( f'''[{i}] {tot}''')
        i += 1
    return tot

In [251]:
for d,g in zip(test['data'], test['groups']):
    print("\n\n\n")
    print(d,g)
    print( count_solutions(d,g, verbose=True) )






???.### [1, 1, 3]
Remaining string: ???.###
[1, 1, 3]
Remaining string: ?.###
[1, 3]
Remaining string: ###
[3]
1




.??..??...?##. [1, 1, 3]
Remaining string: .??..??...?##.
[1, 1, 3]
Remaining string: ..??...?##.
[1, 3]
Remaining string: ...?##.
[3]
Remaining string: ..?##.
[3]
Remaining string: .??...?##.
[1, 3]
Remaining string: ...?##.
[1, 3]
Remaining string: ..?##.
[1, 3]
4




?#?#?#?#?#?#?#? [1, 3, 1, 6]
Remaining string: ?#?#?#?#?#?#?#?
[1, 3, 1, 6]
Remaining string: #?#?#?#?#?#?
[3, 1, 6]
Remaining string: #?#?#?#?
[1, 6]
Remaining string: #?#?#?
[6]
1




????.#...#... [4, 1, 1]
Remaining string: ????.#...#...
[4, 1, 1]
Remaining string: #...#...
[1, 1]
Remaining string: ..#...
[1]
1




????.######..#####. [1, 6, 5]
Remaining string: ????.######..#####.
[1, 6, 5]
Remaining string: ??.######..#####.
[6, 5]
Remaining string: .#####.
[5]
Remaining string: ?.######..#####.
[6, 5]
Remaining string: .#####.
[5]
Remaining string: .######..#####.
[6, 5]
Remaining string: .####

In [252]:
part1(test)

[0] 1
[1] 5
[2] 6
[3] 7
[4] 11
[5] 21


21

In [253]:
# too high 8838
# too high 7615
# too high 7255
# yay! 7173
part1(puzz)

[0] 3
[1] 7
[2] 23
[3] 34
[4] 43
[5] 95
[6] 97
[7] 106
[8] 124
[9] 128
[10] 190
[11] 192
[12] 195
[13] 205
[14] 237
[15] 242
[16] 266
[17] 268
[18] 273
[19] 276
[20] 278
[21] 284
[22] 288
[23] 294
[24] 299
[25] 302
[26] 339
[27] 343
[28] 346
[29] 353
[30] 388
[31] 392
[32] 395
[33] 396
[34] 400
[35] 403
[36] 415
[37] 420
[38] 421
[39] 422
[40] 427
[41] 432
[42] 444
[43] 446
[44] 452
[45] 456
[46] 462
[47] 465
[48] 466
[49] 468
[50] 472
[51] 504
[52] 510
[53] 516
[54] 549
[55] 552
[56] 564
[57] 601
[58] 608
[59] 616
[60] 618
[61] 619
[62] 625
[63] 626
[64] 628
[65] 638
[66] 641
[67] 644
[68] 646
[69] 647
[70] 648
[71] 652
[72] 653
[73] 657
[74] 663
[75] 664
[76] 668
[77] 679
[78] 685
[79] 692
[80] 693
[81] 694
[82] 709
[83] 715
[84] 743
[85] 755
[86] 780
[87] 785
[88] 792
[89] 796
[90] 799
[91] 813
[92] 814
[93] 815
[94] 820
[95] 821
[96] 823
[97] 828
[98] 832
[99] 833
[100] 835
[101] 848
[102] 852
[103] 854
[104] 862
[105] 877
[106] 879
[107] 883
[108] 886
[109] 932
[110] 952
[111] 953

7173

### Part 2

In [254]:
def embiggen_puzzle( puzz ):
    big_puzz = {}

    return {
        'data': [ "?".join( [x]*5 ) for x in puzz['data'] ],
        'groups': [ x*5 for x in puzz['groups'] ]
    }

In [255]:
btest = embiggen_puzzle(test)

In [256]:
btest

{'data': ['???.###????.###????.###????.###????.###',
  '.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.',
  '?#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#?',
  '????.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...',
  '????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.',
  '?###??????????###??????????###??????????###??????????###????????'],
 'groups': [[1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3],
  [1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3],
  [1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6],
  [4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1],
  [1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5],
  [3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1]]}

In [257]:
part1(btest)

[0] 1
[1] 16385
[2] 16386
[3] 16402
[4] 18902
[5] 525152


525152

In [258]:
bpuzz = embiggen_puzzle(puzz)

In [259]:
bpuzz['data'][2], bpuzz['groups'][2]

('.???.??.???.???.??.???.???.??.???.???.??.???.???.??.??',
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [260]:
count_solutions( bpuzz['data'][2], bpuzz['groups'][2], verbose=True )

Remaining string: .???.??.???.???.??.???.???.??.???.???.??.???.???.??.??
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining string: ?.??.???.???.??.???.???.??.???.???.??.???.???.??.??
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining string: ??.???.???.??.???.???.??.???.???.??.???.???.??.??
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining string: .???.???.??.???.???.??.???.???.??.???.???.??.??
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining string: ?.???.??.???.???.??.???.???.??.???.???.??.??
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining string: ???.??.???.???.??.???.???.??.???.???.??.??
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining string: ?.??.???.???.??.???.???.??.???.???.??.??
[1, 1, 1, 1, 1, 1, 1, 1, 1]
Remaining string: ??.???.???.??.???.???.??.???.???.??.??
[1, 1, 1, 1, 1, 1, 1, 1]
Remaining string: .???.???.??.???.???.??.???.???.??.??
[1, 1, 1, 1, 1, 1, 1]
Remaining string: ?.???.??.???.???.??.???.???.??.??
[1, 1, 1, 1, 1, 1]
Remaining string: ???.??.???.???.??.???.???.??.??
[

173257860

In [261]:
# too low: 173257860
part1(bpuzz)

[0] 3888
[1] 32757
[2] 173290617
[3] 173632212
[4] 179160581
[5] 63407947173
[6] 63407947205
[7] 63409074005
[8] 63453078473
[9] 63453079497
[10] 147487745618
[11] 147487745650
[12] 147487745893
[13] 147488436281
[14] 149772541331
[15] 149772558869
[16] 150291771733
[17] 150291771895
[18] 150291778375
[19] 150291778618
[20] 150291778650
[21] 150291786426
[22] 150291793330
[23] 150291845989
[24] 150291871529
[25] 150291871772
[26] 156467546209
[27] 156467547233
[28] 156467547476
[29] 156467649963
[30] 157356373253
[31] 157356374277
[32] 157356374520
[33] 157356375816
[34] 157356378316
[35] 157356397866
[36] 157368964026
[37] 157369024194
[38] 157369024195
[39] 157369024276
[40] 157369085385
[41] 157369088510
[42] 157371899582
[43] 157371899614
[44] 157372873570
[45] 157372878754
[46] 157373370090
[47] 157373370333
[48] 157373371213
[49] 157373371245
[50] 157373372269
[51] 163106085767
[52] 163106117963
[53] 163106125739
[54] 163592713571
[55] 163592715446
[56] 163598378173
[57] 16758468

29826669191291